In [73]:
import pandas as pd
pd.set_option('display.max_columns', None)

orig_df = pd.read_excel("../Judgment_data_9.20.24.xlsx", sheet_name="Comp 3 - All groups w prof")

orig_df = orig_df.iloc[1:]

# need to make a df with columns of PID, group, QID, rating, adverb, adverb class, mente, structure
orig_df = orig_df.rename(columns={'P#': 'PID'})


def get_adverb_class(qid: str):
    type_key = {
        "A": "Place",
        "B": "Extent",
        "C": "Manner",
        "D": "Quantity"
    }

    return type_key[qid.split("_")[1][1]]


def get_structure(qid: str):
    structure_key = qid.split("_")[1][0]

    d = {
        "A": "AdvSVO",
        "B": "SAdvVO",
        "C": "SVAdvO",
        "D": "SVOAdv"
    }

    return d[structure_key]


headers = {
    "PID": [],
    "Group": [],
    "QID": [],
    "Rating": [],
    "Adverb": [],
    "Mente": [],
    "Adverb_Class": [],
    "Structure": []
}

# Now, we go through each person and make a row for each question, PID combination
for pid, row in orig_df.iterrows():
    group = row["Group"]

    for col in orig_df.columns:
        if "AJT" in col:
            qid = col
            mente = 'Yes' if qid.split("_")[1][-1] == "+" else 'No'
            adverb = qid.split("_")[-1]

            rating = row[col]
            # Add the current data as a row to the stats df

            headers["PID"].append(pid)
            headers["Group"].append(group)
            headers["QID"].append(qid)
            headers["Rating"].append(rating)
            headers["Adverb"].append(adverb)
            headers["Mente"].append(mente)
            headers["Adverb_Class"].append(get_adverb_class(qid))
            headers["Structure"].append(get_structure(qid))

df = pd.DataFrame.from_dict(headers)
df.to_csv('ajt_data.csv', index=False)

df['Group'] = pd.Categorical(df['Group'], df['Group'].unique().tolist())
df['Adverb'] = pd.Categorical(df['Adverb'], df['Adverb'].unique().tolist())
df['Adverb_Class'] = pd.Categorical(df['Adverb_Class'], df['Adverb_Class'].unique().tolist())
df['Rating'] = df['Rating'].astype(int)

print(df)

       PID Group                   QID  Rating       Adverb Mente  \
0        1    NS         AJT0_AA-_aqui       4         aqui    No   
1        1    NS         AJT0_BA-_aqui       1         aqui    No   
2        1    NS         AJT0_CA-_aqui       2         aqui    No   
3        1    NS         AJT0_DA-_aqui       2         aqui    No   
4        1    NS       AJT0_AA-_afuera       4       afuera    No   
...    ...   ...                   ...     ...          ...   ...   
12539   98   Low         AJT1_DD-_poco       4         poco    No   
12540   98   Low  AJT1_AD+_minimamente       2  minimamente   Yes   
12541   98   Low  AJT1_BD+_minimamente       3  minimamente   Yes   
12542   98   Low  AJT1_CD+_minimamente       3  minimamente   Yes   
12543   98   Low  AJT1_DD+_minimamente       4  minimamente   Yes   

      Adverb_Class Structure  
0            Place    AdvSVO  
1            Place    SAdvVO  
2            Place    SVAdvO  
3            Place    SVOAdv  
4            Pla

In [74]:
# get the number of participants in each group
writer = pd.ExcelWriter("ajt_processed_data.xlsx", engine='xlsxwriter')

df.groupby(['Group']).agg('count')['PID'] / df['QID'].nunique()

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/2146691376.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Group']).agg('count')['PID'] / df['QID'].nunique()


Group
NS         19.0
HS Low      8.0
HS Int     13.0
HS High    10.0
High        8.0
Int        19.0
Low        21.0
Name: PID, dtype: float64

In [75]:
# Get the average rating for each group
ratings_df = pd.pivot_table(df, values="Rating", index="Group")
ratings_df.to_excel(writer, sheet_name='Overall Ratings')
ratings_df

,Rating
Group,
NS,2.543174
HS Low,2.665039
HS Int,2.687500
HS High,2.756250
High,2.681641
Int,2.664474
Low,2.533110


In [76]:
# Get the average rating for each adverb
adv_ratings_df = pd.pivot_table(df, values="Rating", index=["Group", "Adverb"]).unstack()
adv_ratings_df.to_excel(writer, sheet_name='Adverb Ratings')
adv_ratings_df

Rating                                                          \
Adverb       aqui    afuera     cerca     lejos      casi aproximadamente   
Group                                                                       
NS       3.019737  2.894737  2.006579  1.776316  2.217105        2.572368   
HS Low   2.750000  2.781250  2.328125  2.296875  2.640625        2.875000   
HS Int   3.115385  2.817308  2.288462  2.134615  2.423077        2.817308   
HS High  3.300000  2.800000  2.187500  2.212500  2.387500        3.112500   
High     3.078125  2.843750  2.312500  2.328125  1.984375        2.859375   
Int      2.881579  2.618421  2.230263  2.296053  2.447368        3.065789   
Low      2.726190  2.553571  2.363095  2.261905  2.357143        2.750000   

                                                                         \
Adverb     apenas solamente facilmente      bien       mal precisamente   
Group                                                                     
NS       2.671053  3.421053   3.388158  2.532895  2.309211     2.717105   
HS Low   2.812500  2.953125   3.203125  2.734375  2.140625     3.125000   
HS Int   2.615385  3.259615   3.211538  2.701923  2.038462     3.038462   
HS High  2.400000  3.275000   3.475000  2.762500  2.112500     3.262500   
High     2.203125  3.328125   3.265625  2.593750  2.296875     3.062500   
Int      2.184211  3.164474   3.111842  2.513158  2.151316     3.151316   
Low      2.202381  3.011905   2.750000  2.351190  2.136905     2.940476   

                                                    
Adverb      mucho mayormente      poco minimamente  
Group                                               
NS       1.953947   2.921053  1.921053    2.368421  
HS Low   2.359375   2.750000  2.359375    2.531250  
HS Int   2.461538   2.923077  2.317308    2.836538  
HS High  2.287500   3.037500  2.500000    2.987500  
High     2.250000   3.218750  2.406250    2.875000  
Int      2.368421   3.072368  2.532895    2.842105  
Low      2.404762   2.720238  2.386905    2.613095

In [77]:
# Split these into each structure
import math

adv_ratings_per_struct_df = df.groupby(["Adverb", "Group", "Structure"]).apply(
    lambda x: pd.Series({
        'mean': x['Rating'].mean(),
        'count': x['Rating'].count(),
        'std': x['Rating'].std()
    })
)

ci_95_variance = []

for index, row in adv_ratings_per_struct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_ratings_per_struct_df['ci95_variance'] = ci_95_variance
adv_ratings_per_struct_df.drop(columns=['count', 'std'], inplace=True)
adv_ratings_per_struct_df = adv_ratings_per_struct_df.unstack(level=2)

adv_ratings_per_struct_df.to_excel(writer, sheet_name='Adverb Rating Per Struct')
adv_ratings_per_struct_df

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/788606558.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adv_ratings_per_struct_df = df.groupby(["Adverb", "Group", "Structure"]).apply(


mean                               ci95_variance  \
Structure              AdvSVO    SAdvVO    SVAdvO    SVOAdv        AdvSVO   
Adverb      Group                                                           
aqui        NS       3.342105  2.473684  2.736842  3.526316      0.279310   
            HS Low   3.437500  2.375000  2.562500  2.625000      0.398831   
            HS Int   3.461538  2.807692  2.576923  3.615385      0.380226   
            HS High  3.800000  2.500000  2.900000  4.000000      0.229280   
            High     3.500000  2.500000  2.375000  3.937500      0.357845   
...                       ...       ...       ...       ...           ...   
minimamente HS Int   2.192308  3.115385  2.884615  3.153846      0.435146   
            HS High  2.100000  3.350000  3.050000  3.450000      0.373511   
            High     1.937500  2.625000  3.312500  3.625000      0.418417   
            Int      2.105263  2.921053  2.710526  3.631579      0.337055   
            Low      2.119048  2.761905  2.047619  3.523810      0.321752   

                                                   
Structure              SAdvVO    SVAdvO    SVOAdv  
Adverb      Group                                  
aqui        NS       0.319983  0.359756  0.230668  
            HS Low   0.562256  0.472327  0.533028  
            HS Int   0.461511  0.437231  0.289226  
            HS High  0.460759  0.399662  0.000000  
            High     0.506070  0.395051  0.122500  
...                       ...       ...       ...  
minimamente HS Int   0.411520  0.425639  0.387333  
            HS High  0.356194  0.502226  0.332714  
            High     0.562256  0.345036  0.303378  
            Int      0.289920  0.312824  0.226897  
            Low      0.281891  0.249505  0.260747  

[112 rows x 8 columns]

In [78]:
# Get the averages for each adverb type
adv_class_rating_df = df.groupby(["Group", "Adverb_Class"]).apply(
    lambda x: pd.Series({
        'mean': x['Rating'].mean(),
        'count': x['Rating'].count(),
        'std': x['Rating'].std()
    })
)

ci_95_variance = []

for index, row in adv_class_rating_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_class_rating_df['ci95_variance'] = ci_95_variance
adv_class_rating_df.drop(columns=['count', 'std'], inplace=True)
adv_class_rating_df = adv_class_rating_df.unstack()

adv_class_rating_df.to_excel(writer, sheet_name='Adverb Class Ratings')
adv_class_rating_df.to_clipboard()
adv_class_rating_df

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/2808650767.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adv_class_rating_df = df.groupby(["Group", "Adverb_Class"]).apply(


mean                               ci95_variance            \
Adverb_Class     Place    Extent    Manner  Quantity         Place    Extent   
Group                                                                          
NS            2.424342  2.720395  2.736842  2.291118      0.093759  0.095079   
HS Low        2.539062  2.820312  2.800781  2.500000      0.131447  0.125989   
HS Int        2.588942  2.778846  2.747596  2.634615      0.121252  0.112982   
HS High       2.625000  2.793750  2.903125  2.703125      0.125572  0.120898   
High          2.640625  2.593750  2.804688  2.687500      0.136999  0.151446   
Int           2.506579  2.715461  2.731908  2.703947      0.087764  0.088226   
Low           2.476190  2.580357  2.544643  2.531250      0.088899  0.085081   

                                  
Adverb_Class    Manner  Quantity  
Group                             
NS            0.096245  0.093256  
HS Low        0.136117  0.136367  
HS Int        0.121600  0.117501  
HS High       0.130134  0.124034  
High          0.149979  0.148156  
Int           0.096105  0.088869  
Low           0.093433  0.086947

In [79]:
adv_class_rating_per_struct_df = df.groupby(["Adverb_Class", "Group", "Structure"]).apply(
    lambda x: pd.Series({
        'mean': x['Rating'].mean(),
        'count': x['Rating'].count(),
        'std': x['Rating'].std()
    })
)

ci_95_variance = []

for index, row in adv_class_rating_per_struct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

adv_class_rating_per_struct_df['ci95_variance'] = ci_95_variance
adv_class_rating_per_struct_df.drop(columns=['count', 'std'], inplace=True)
adv_class_rating_per_struct_df = adv_class_rating_per_struct_df.unstack()

adv_class_rating_per_struct_df.to_excel(writer, sheet_name='Adverb Class Rating Per Struct')
adv_class_rating_per_struct_df

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/469844031.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adv_class_rating_per_struct_df = df.groupby(["Adverb_Class", "Group", "Structure"]).apply(


mean                               ci95_variance  \
Structure               AdvSVO    SAdvVO    SVAdvO    SVOAdv        AdvSVO   
Adverb_Class Group                                                           
Place        NS       2.342105  1.809211  2.342105  3.203947      0.186680   
             HS Low   2.875000  2.171875  2.359375  2.750000      0.270858   
             HS Int   2.682692  2.048077  2.269231  3.355769      0.254445   
             HS High  2.712500  1.975000  2.325000  3.487500      0.251896   
             High     2.843750  1.781250  2.234375  3.703125      0.269867   
             Int      2.592105  1.894737  2.026316  3.513158      0.170022   
             Low      2.642857  1.857143  1.833333  3.571429      0.182248   
Extent       NS       2.019737  3.230263  3.151316  2.480263      0.167158   
             HS Low   2.843750  3.265625  2.843750  2.328125      0.235962   
             HS Int   2.326923  3.519231  2.836538  2.432692      0.216653   
             HS High  2.362500  3.550000  2.775000  2.487500      0.206992   
             High     1.828125  3.312500  2.875000  2.359375      0.252864   
             Int      2.144737  3.269737  2.789474  2.657895      0.166091   
             Low      2.119048  2.791667  2.380952  3.029762      0.155070   
Manner       NS       1.750000  2.236842  3.440789  3.519737      0.150040   
             HS Low   2.359375  2.500000  3.203125  3.140625      0.277348   
             HS Int   1.855769  2.519231  3.144231  3.471154      0.209887   
             HS High  2.025000  2.587500  3.312500  3.687500      0.217674   
             High     1.750000  2.328125  3.546875  3.593750      0.239096   
             Int      1.802632  2.519737  2.986842  3.618421      0.158491   
             Low      1.976190  2.232143  2.321429  3.648810      0.174312   
Quantity     NS       1.611842  2.013158  3.019737  2.519737      0.139399   
             HS Low   2.187500  2.500000  2.625000  2.687500      0.275652   
             HS Int   1.875000  2.509615  3.115385  3.038462      0.203412   
             HS High  1.950000  2.537500  3.087500  3.237500      0.211800   
             High     1.781250  2.312500  3.187500  3.468750      0.252540   
             Int      1.947368  2.598684  2.842105  3.427632      0.159281   
             Low      1.898810  2.440476  2.386905  3.398810      0.157550   

                                                    
Structure               SAdvVO    SVAdvO    SVOAdv  
Adverb_Class Group                                  
Place        NS       0.142585  0.181220  0.167026  
             HS Low   0.249068  0.255907  0.243048  
             HS Int   0.218200  0.216303  0.202051  
             HS High  0.197160  0.222643  0.203288  
             High     0.202262  0.208174  0.142656  
             Int      0.134645  0.141660  0.124073  
             Low      0.132683  0.135118  0.106698  
Extent       NS       0.166154  0.175262  0.183848  
             HS Low   0.196399  0.251595  0.269287  
             HS Int   0.165051  0.235176  0.208766  
             HS High  0.193817  0.248938  0.228621  
             High     0.242557  0.274353  0.312863  
             Int      0.141687  0.167319  0.182142  
             Low      0.165364  0.165327  0.161656  
Manner       NS       0.179020  0.151686  0.109732  
             HS Low   0.292831  0.251447  0.199408  
             HS Int   0.248334  0.211588  0.175530  
             HS High  0.282602  0.216606  0.137685  
             High     0.289739  0.184841  0.188864  
             Int      0.190992  0.172593  0.111192  
             Low      0.175892  0.160718  0.119493  
Quantity     NS       0.171621  0.174985  0.178301  
             HS Low   0.265529  0.270858  0.268650  
             HS Int   0.239534  0.202741  0.214124  
             HS High  0.253700  0.229152  0.191748  
             High     0.295664  0.226300  0.204604  
             Int      0.174784  0.158561  0.131430  
             Low 

In [80]:
# Look at mente overall
mente_df = df.groupby(["Group", "Mente"]).apply(
    lambda x: pd.Series({
        'mean': x['Rating'].mean(),
        'count': x['Rating'].count(),
        'std': x['Rating'].std()
    })
)

ci_95_variance = []

for index, row in mente_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

mente_df['ci95_variance'] = ci_95_variance
mente_df.drop(columns=['count', 'std'], inplace=True)
mente_df = mente_df.unstack()

mente_df.to_excel(writer, sheet_name='Mente')
mente_df

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/2181136596.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mente_df = df.groupby(["Group", "Mente"]).apply(


mean           ci95_variance          
Mente          No       Yes            No       Yes
Group                                              
NS       2.330263  2.898026      0.061810  0.069595
HS Low   2.520313  2.906250      0.087751  0.097055
HS Int   2.491346  3.014423      0.077603  0.084868
HS High  2.495000  3.191667      0.082336  0.082400
High     2.429688  3.101562      0.093408  0.106354
Int      2.422368  3.067982      0.058186  0.063853
Low      2.374405  2.797619      0.055962  0.069621

In [81]:
# Split by structure
mente_in_structs_df = pd.pivot_table(df, values="Rating", index=["Mente", "Group", "Structure"]).unstack(level=1)
mente_in_structs_df.to_excel(writer, sheet_name='Mente in Structs')
mente_in_structs_df

Rating                                                    \
Group                  NS    HS Low    HS Int   HS High      High       Int   
Mente Structure                                                               
No    AdvSVO     1.781579  2.437500  2.046154  2.065000  1.931250  1.955263   
      SAdvVO     1.997368  2.368750  2.192308  2.165000  2.056250  2.157895   
      SVAdvO     2.710526  2.668750  2.650000  2.620000  2.587500  2.381579   
      SVOAdv     2.831579  2.606250  3.076923  3.130000  3.143750  3.194737   
Yes   AdvSVO     2.179825  2.781250  2.416667  2.591667  2.250000  2.399123   
      SAdvVO     2.864035  3.010417  3.410256  3.491667  3.062500  3.258772   
      SVAdvO     3.451754  2.906250  3.160256  3.300000  3.583333  3.127193   
      SVOAdv     3.096491  2.927083  3.070513  3.383333  3.510417  3.486842   

                           
Group                 Low  
Mente Structure            
No    AdvSVO     2.040476  
      SAdvVO     2.054762  
      SVAdvO     2.059524  
      SVOAdv     3.342857  
Yes   AdvSVO     2.357143  
      SAdvVO     2.789683  
      SVAdvO     2.515873  
      SVOAdv     3.527778

In [82]:
ratings_by_stuct_df = df.groupby(["Structure", "Group"]).apply(
    lambda x: pd.Series({
        'mean': x['Rating'].mean(),
        'count': x['Rating'].count(),
        'std': x['Rating'].std()
    })
)

ci_95_variance = []

for index, row in ratings_by_stuct_df.iterrows():
    mean, count, std = (row["mean"], row['count'], row['std'])
    
    ci_95_variance.append(1.96*std/math.sqrt(count))

ratings_by_stuct_df['ci95_variance'] = ci_95_variance
ratings_by_stuct_df.drop(columns=['count', 'std'], inplace=True)
ratings_by_stuct_df = ratings_by_stuct_df.unstack()
ratings_by_stuct_df.to_excel(writer, sheet_name='Structs')
ratings_by_stuct_df

/var/folders/_r/7zvxypfd3c92zxk5f_c5f6080000gn/T/ipykernel_56318/3784393902.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ratings_by_stuct_df = df.groupby(["Structure", "Group"]).apply(


mean                                                            \
Group            NS    HS Low    HS Int HS High      High       Int       Low   
Structure                                                                       
AdvSVO     1.930921  2.566406  2.185096  2.2625  2.050781  2.121711  2.159226   
SAdvVO     2.322368  2.609375  2.649038  2.6625  2.433594  2.570724  2.330357   
SVAdvO     2.988487  2.757812  2.841346  2.8750  2.960938  2.661184  2.230655   
SVOAdv     2.930921  2.726562  3.074519  3.2250  3.281250  3.304276  3.412202   

          ci95_variance                                                    \
Group                NS    HS Low    HS Int   HS High      High       Int   
Structure                                                                   
AdvSVO         0.083705  0.136985  0.115427  0.115739  0.138155  0.084933   
SAdvVO         0.093219  0.135270  0.121257  0.132364  0.146131  0.089699   
SVAdvO         0.091193  0.133529  0.113167  0.121304  0.126891  0.085373   
SVOAdv         0.088422  0.127643  0.107226  0.108324  0.128307  0.075988   

                     
Group           Low  
Structure            
AdvSVO     0.086486  
SAdvVO     0.083617  
SVAdvO     0.079038  
SVOAdv     0.069000

In [83]:
writer.close()